In [1]:
import pandas as pd
import numpy as np
import pyreadstat as st
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [2]:
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2024 3T\[MT06] Microeconometría I\Clases\Stata\Archivos Stata-20250107\laborsub.dta"

df, meta = st.read_dta(path)
df.head()

,lfp,whrs,kl6,k618,wa,we
0,1,1610,1,0,32,12
1,1,1656,0,2,30,12
2,1,1980,1,3,35,12
3,1,456,0,3,34,12
4,1,1568,1,2,31,14


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   lfp     250 non-null    int64
 1   whrs    250 non-null    int64
 2   kl6     250 non-null    int64
 3   k618    250 non-null    int64
 4   wa      250 non-null    int64
 5   we      250 non-null    int64
dtypes: int64(6)
memory usage: 11.8 KB


In [4]:
meta.column_labels

['1 if woman worked in 1975',
 "Wife's hours of work",
 '# of children younger than 6',
 '# of children between 6 and 18',
 "Wife's age",
 "Wife's educational attainment"]

In [5]:
df['lfp'].value_counts()

lfp
1    150
0    100
Name: count, dtype: int64

In [9]:
df[['whrs']].describe().T

,count,mean,std,min,25%,50%,75%,max
whrs,250.0,799.84,915.60348,0.0,0.0,406.5,1599.75,4950.0
